In [3]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import os,sys

In [4]:
import re
from PIL import Image
import glob

### API

In [18]:
chi_label2id8 = {
    0: '鞋子',
    1: '包包',
    2: '上装_短款',
    3: '裤子_短款',
    4: '裙子',
    5: '连体装',
    6: '上装_长款',
    7: '裤子_长款',
}

In [19]:
eng_label2id8 = {
    0: 'shoe',
    1: 'bag',
    2: 'upperbody_short',
    3: 'lowerbody_short',
    4: 'shirt',
    5: 'wholebody',
    6: 'upperbody_long',
    7: 'lowerbody_long',
}

In [12]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [ ]:
id2label_chi8 = get_list2dict(chi_label2id8.values(), chi_label2id8.keys())

### information

In [7]:
root = Path('/nas/chenyi/datasets_nas/dada_det/xml/wardrobe500k/wardrobe20k_xml_hard20220705/')
root_csv = root /'info_csv/xml_clr_20220705.csv'
xml_df = pd.read_csv(root_csv)
len(xml_df)

20971

In [8]:
!ls $root/

info_csv  xml


In [9]:
xml_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp,label86
0,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,1080,1554,裙子,6,478,1080,1129,2017022320122485968,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,4


In [20]:
# xml_df['label_eng'] = xml_df['label86'].map(eng_label2id8)

#### one object.

In [ ]:
bboxes_temp = xml_df

In [56]:
bboxes_temp['proportion'] = bboxes_temp['bx3'].values*bboxes_temp['bx4'].values

In [57]:
# bbox占图比例,小于阈值取最大
thres = 1/4
bboxes_large = bboxes_temp[bboxes_temp['proportion']>thres]
bboxes_small = bboxes_temp[~bboxes_temp['stem'].isin(bboxes_large['stem'])]
bboxes_small.sort_values('proportion', ascending=False, inplace=True)
bboxes_small.drop_duplicates('stem', keep='first', inplace=True)
bboxes_temp = pd.concat([bboxes_large, bboxes_small], axis=0)

/tmp/ipykernel_2866568/3187007672.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bboxes_small.sort_values('proportion', ascending=False, inplace=True)


In [55]:
# 计算距离
bboxes_temp['distance_abs'] = np.sqrt(((bboxes_temp['bx1'].values-0.5)*bboxes_temp['width'].values)**2 + 
                              ((bboxes_temp['bx2'].values-0.5)*bboxes_temp['height'].values)**2)

In [58]:
# 取距离中心最近的
bboxes_temp.sort_values('distance_abs', ascending=True, inplace=True)
bboxes_temp = bboxes_temp.drop_duplicates('stem', keep='first')

In [63]:
len(bboxes_temp), 

(20361,)

#### 生成target path

In [65]:
xml_df = bboxes_temp

In [66]:
root_xml = root /'xml_'
root_img = root /'images_'

In [12]:
for item in xml_df.value_counts('labelimg').index:
    print(item)

上装_短款
裙子
裤子_长款
连体装
裤子_短款
包包
鞋子
上装_长款


In [145]:
cat_xml_df = xml_df[xml_df['label_eng']=='upperbody_long']

In [146]:
cat_xml_df.reset_index(inplace=True, drop=True)

In [147]:
cat_xml_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp,label86,label_eng,distance_abs,proportion
0,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,400,880,上装_长款,2,1,397,879,2017072509571102016,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,6,upperbody_long,744.04301,348963


In [148]:
cat_xml_df['index'] = cat_xml_df.index.values

/tmp/ipykernel_2866568/2611509796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_xml_df['index'] = cat_xml_df.index.values


In [149]:
folder_num = 255
cat_xml_df['xml_tp'] = cat_xml_df.apply(lambda x: '{0}/{1}/{2}/{3}.xml'.format(root_xml, x['label_eng'], x['index']//folder_num, x['stem']), axis=1)

/tmp/ipykernel_2866568/2374103287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_xml_df['xml_tp'] = cat_xml_df.apply(lambda x: '{0}/{1}/{2}/{3}.xml'.format(root_xml, x['label_eng'], x['index']//folder_num, x['stem']), axis=1)


In [150]:
cat_xml_df['img_tp'] = cat_xml_df.apply(lambda x: '{0}/{1}/{2}/{3}.jpg'.format(root_img, x['label_eng'], x['index']//folder_num, x['stem']), axis=1)

/tmp/ipykernel_2866568/1668441929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_xml_df['img_tp'] = cat_xml_df.apply(lambda x: '{0}/{1}/{2}/{3}.jpg'.format(root_img, x['label_eng'], x['index']//folder_num, x['stem']), axis=1)


In [151]:
cat_xml_df['xml_tp'].values[1]

'/nas/chenyi/datasets_nas/wardrobe20k_labelimg/xml_/upperbody_long/0/2017032415200631960.xml'

#### 数据拷贝

In [152]:
# xml_check_dir = root_xml
# if Path(xml_check_dir).is_dir():
#     shutil.rmtree(xml_check_dir)
# Path.mkdir(xml_check_dir, parents=True)

In [153]:
# img_check_dir = root_img
# if Path(img_check_dir).is_dir():
#     shutil.rmtree(img_check_dir)
# Path.mkdir(img_check_dir, parents=True)

In [154]:
import shutil
from tqdm import tqdm
for ind,row in tqdm(cat_xml_df.iterrows()):
    sp = Path(row['xml_path'])
    tp = Path(row['xml_tp'])
    if not tp.parent.is_dir():
        tp.parent.mkdir(exist_ok=True, parents=True)
    shutil.copyfile(sp, tp)
#     shutil.move(sp, tp)

212it [00:01, 120.30it/s]


In [155]:
import shutil
from tqdm import tqdm
for ind,row in tqdm(cat_xml_df.iterrows()):
    sp = Path(row['img_sp'])
    tp = Path(row['img_tp'])
    if not tp.parent.is_dir():
        tp.parent.mkdir(exist_ok=True, parents=True)
    shutil.copyfile(sp, tp)
#     shutil.move(sp, tp)

212it [00:02, 86.27it/s] 


##### 删除多余xml

In [159]:
root_xml

PosixPath('/nas/chenyi/datasets_nas/wardrobe20k_labelimg/xml_')

In [162]:
xml_list = glob.glob(str(root_xml /'*/*/*.xml'))

In [163]:
xml_folder_df = pd.DataFrame()
xml_folder_df['xml_sp'] = xml_list

In [164]:
xml_folder_df.head(1)

,xml_sp
0,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...


In [165]:
xml_folder_df['stem'] = xml_folder_df['xml_sp'].apply(lambda x: Path(x).stem)

In [171]:
surplus_df = xml_folder_df[~xml_folder_df['stem'].isin(xml_df['stem'])]

In [172]:
len(surplus_df), len(xml_df), len(xml_folder_df)

(414, 20971, 20361)

In [175]:
import shutil
from tqdm import tqdm
for p in tqdm(surplus_df['xml_sp'].values):
    sp = Path(p)
    if sp.is_file():
        os.remove(sp)
    # break

100%|██████████████████████████████████████████████████████████████████████████████████████| 414/414 [00:00<00:00, 2724.35it/s]
